# 🛠️ Notebook: Tool Usage with Conversational Agents

This notebook showcases how an agent can use external tools—like currency conversion and environmental estimations—by reasoning about when to call them during a conversation.


## 📦 Import Modules

Loads the necessary modules for agent behavior and user simulation, including custom user agents and system modules.


In [ ]:
from agents import Agent, User
import os

## 🧠 Define Agent Tools and Create Agent

Defines custom tools (`currency_calculator`, `exchange_rate`, and `tree_estimator`) that the agent can call during conversation. Then, sets up the assistant agent and user proxy agent with tool-calling capabilities and detailed system instructions.


In [ ]:
# Ollama Client
# llm_config = {
#     "config_list": [
#         {
#             "client": "ollama",
#             "temperature": 0.7,
#             "model": "llama3.2",
#             "base_url": "http://localhost:11434",
#         }
#     ]
# }
gpt_41 = {
    "config_list": [
        {
            "client": "openai",
            "temperature": 0.0,
            "model": "gpt-4.1",
            "api_key": os.getenv("OPENAI_API_KEY"),
        }
    ]
}
llm_config = gpt_41
from typing import Literal

Operator = Literal["+", "-", "*", "/"]

def exchange_rate(base_currency: str, quote_currency: str) -> float:
    """
    Returns the exchange rate between two currencies.

    Parameters:
    - base_currency (str): The currency you want to convert from (e.g., "USD").
    - quote_currency (str): The currency you want to convert to (e.g., "EUR").

    Returns:
    - float: The exchange rate from base_currency to quote_currency.

    Raises:
    - ValueError: If the currency pair is unknown or not supported.
    """

    # If the base and quote currencies are the same, return 1.0
    if base_currency == quote_currency:
        return 1.0

    # Conversion from USD to EUR (example rate)
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.09

    # Conversion from EUR to USD (example rate)
    elif base_currency == "EUR" and quote_currency == "USD":
        return 5.1

    # Raise error if the currency pair is not recognized
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")

def currency_calculator(
        base_amount: float,
        base_currency:str,
        quote_currency:str,
) -> str:
    """
    Converts a monetary amount from one currency to another.

    Parameters:
    - base_amount (float): The amount of money in the base currency.
    - base_currency (str): The currency in which the base_amount is expressed.
    - quote_currency (str): The target currency to convert the base_amount into.

    Returns:
    - str: A string representing the converted amount followed by the quote currency.
    """

    # Get the converted amount using the exchange rate
    quote_amount = exchange_rate(base_currency, quote_currency) * float(base_amount)

    # Return the converted amount as a formatted string
    return f"{quote_amount} {quote_currency}"


def tree_estimator(
        region: str = "Earth",
        unit: str = "trees"
) -> str:
    """
    Estimates the number of trees on a given region (default is Earth).

    Parameters:
    - region (str): The geographical area to estimate tree count (default is "Earth").
    - unit (str): The unit of measurement to return (default is "trees").

    Returns:
    - str: A string representing the estimated number of trees.
    """

    # Static estimate based on global scientific studies
    if region.lower() == "earth":
        estimated_trees = 1
    else:
        estimated_trees = 0

    return f"{estimated_trees:,} {unit}"

# Let's first define the assistant agent that suggests tool calls.
user = User(
    name="user"
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
math = Agent(
    name="math",
    llm_config=llm_config,
    system_message=
        "You never execute functions yourself — you only suggest them.\n"
        "If multiple functions are needed (e.g., tree_estimator then currency_calculator), include ALL tool_calls in the same response, in order.\n"
        "Each tool_call must have function.name and function.arguments.\n"
        "Make a brief summary of results after the tool_calls.\n"
        "If the request is unrelated to currency or trees, politely refuse."
,
    tools = [currency_calculator, tree_estimator],
    reflect_on_tool_use=True,
)

## 💬 Simulate User Interaction

The user sends a message to the assistant agent.


In [ ]:
chat_result = user.talk_to(math, message="""
Converta 1 dólar para euro. Converta 1 euro em dollar. Quantas árvores existem na Terra? Responda utilizando suas tools.
""")
